## Week 2 Day 3

Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
# import sendgrid
import os
# from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

import brevo_python 
import asyncio 
import brevo_python 
from brevo_python.api import transactional_emails_api 
from brevo_python.models.send_smtp_email import SendSmtpEmail 
from brevo_python.models.send_smtp_email_to import SendSmtpEmailTo 
from brevo_python.models.send_smtp_email_sender import SendSmtpEmailSender 

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-5okwQ
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### It's easy to use any models with OpenAI compatible endpoints

In [5]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [6]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [7]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [8]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [9]:
@function_tool
def send_html_email(body: str):
    """ Send out an email with the given subject and HTML body to all sales prospects """
    configuration = brevo_python.Configuration()
    api_key = os.environ.get('BREVO_API_KEY')
    if not api_key:
        raise ValueError("BREVO_API_KEY environment variable not set.")
    configuration.api_key['api-key'] = api_key
    api_instance = transactional_emails_api.TransactionalEmailsApi(brevo_python.ApiClient(configuration))
    sender = SendSmtpEmailSender(email="oliver@oliverdreger.cloud", name="Oliver Dreger")
    to = [SendSmtpEmailTo(email="oliver.dreger@gmail.com", name="Oliver Dreger")]
    send_smtp_email = SendSmtpEmail(
        sender=sender,
        to=to,
        subject="Sales email",
        text_content=body
    )
    try:
        api_response = api_instance.send_transac_email(send_smtp_email)
        print("Email sent successfully!")
        print(f"Response: {api_response}")
    except brevo_python.ApiException as e:
        print(f"Exception when calling TransactionalEmailsApi->send_transac_email: {e}\n")
    return {"status": "success"}

In [10]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [11]:
email_tools = [subject_tool, html_tool, send_html_email]

In [12]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [13]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [14]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

Email sent successfully!
Response: {'message_id': '<202507242320.72557531278@smtp-relay.mailin.fr>',
 'message_ids': None}


## Check out the trace:

https://platform.openai.com/traces

In [15]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [16]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [17]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

## Check out the trace:

https://platform.openai.com/traces

In [18]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

Email sent successfully!
Response: {'message_id': '<202507242340.87297407173@smtp-relay.mailin.fr>',
 'message_ids': None}


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>

--- 

# Notebook Summary by Gemini

---

Of course. Here is a summary of the Jupyter Notebook `/Users/oliverdreger/Documents/mygit/GitHub/agents/2_openai/community_contributions/3_lab3_oliver.ipynb`.

## Summary of the Notebook
This Jupyter Notebook is a hands-on lab that demonstrates how to build a sophisticated, multi-agent system for generating and sending sales emails. It explores three key advanced concepts: using multiple Large Language Models (LLMs), ensuring reliable structured outputs, and implementing safety guardrails. 

**1. Multi-Model Integration:** The notebook begins by setting up API keys for various services (OpenAI, Google, DeepSeek, Groq). It then shows how to integrate models from different providers that offer OpenAI-compatible API endpoints.

* It creates custom `AsyncOpenAI` clients for DeepSeek, Google (Gemini), and Groq (for Llama 3.3) by specifying their unique base URLs and API keys. 
* These clients are used to initialize `OpenAIChatCompletionsModel` objects. 
* Three distinct sales agents are created, each with a different persona (professional, humorous, concise) and powered by a different model (DeepSeek, Gemini, Llama 3.3 respectively).  

**2. Agent Orchestration and Workflow:** A multi-layered agent system is constructed to automate the process of writing and sending a cold sales email.

* The three sales agents are converted into tools that a manager agent can call.
* Helper agents are created to handle specific sub-tasks: one for writing an email subject (`subject_writer`) and another for converting the email body to HTML (`html_converter`).
* An `emailer_agent` is defined to orchestrate the formatting (subject and HTML conversion) and sending of the final email, using a `send_html_email` tool that leverages the SendGrid API.
* A top-level `sales_manager` agent is tasked with using the three sales agent tools, evaluating their outputs, selecting the best one, and then handing it off to the `emailer_agent` for final processing and sending.

**3. Structured Outputs and Guardrails:** The final part of the notebook introduces safety and reliability features.

* **Structured Output:** It demonstrates how to force an agent's output into a specific format by defining a Pydantic `BaseModel` (`NameCheckOutput`). This ensures the output is a predictable, parsable object rather than free-form text.
* **Input Guardrail:** An input guardrail is implemented to prevent the system from processing requests that contain personal names. 
    * A dedicated `guardrail_agent` is created. Its sole purpose is to check the user's message for a name. It uses the `NameCheckOutput` Pydantic model to return a structured result. 
    * An `async` function decorated with `@input_guardrail` runs this check. If a name is detected, it triggers a "tripwire," which halts the entire operation and raises an exception. 
* A new `careful_sales_manager` agent is created, identical to the original but with the new input guardrail applied. The notebook concludes by running this "careful" agent twice: once with a prompt containing a name (which fails as expected) and once with a generic title (which succeeds), demonstrating the guardrail's effectiveness.
The notebook ends with an exercise section prompting the user to experiment further with different models, add more guardrails, and use structured outputs for the main email generation.

--- 

# Transcript Summary by Gemini

---

Of course. Here is a detailed summary of Day 3 of the "Master AI Agents in 30 days" course.

---

### **Summary: "Master AI Agents in 30 days" - Day 3**

Day 3 builds upon the Sales Development Representative (SDR) agent project from the previous day, introducing three advanced and critical concepts: integrating multiple LLM providers, implementing guardrails for safety, and using structured outputs for reliability.

#### **1. Recap of Day 2**

The session began with a quick review of Day 2's topics:
*   **Tools:** Using the `@tool` decorator to easily wrap Python functions for agent use.
*   **Agents as Tools:** Converting an entire agent into a tool using the `as_tool()` function.
*   **Handoffs vs. Tools:** Clarifying the difference: a tool call is like a function call that returns a value, while a handoff is a complete transfer of control to another agent in the workflow.

---

#### **2. Multi-Model Integration**

The first major topic was extending the OpenAI Agents SDK to use models from other providers like Google (Gemini), DeepSeek, and Grok (Llama 3).

*   **The Key Requirement:** This integration is straightforward for any model provider that offers an **OpenAI-compatible endpoint**.
*   **Implementation Steps:**
    1.  **Set Environment Variables:** Load API keys for the desired services (e.g., `GOOGLE_API_KEY`, `DEEPSEEK_API_KEY`, `GROQ_API_KEY`).
    2.  **Define Base URLs:** Specify the `base_url` for each provider's OpenAI-compatible endpoint.
    3.  **Create Custom Clients:** For each provider, instantiate an `AsyncOpenAI` client, passing in its specific `base_url` and `api_key`.
    4.  **Create Model Objects:** Instead of using a model name string (e.g., `"gpt-4-mini"`), create an instance of `OpenAIChatCompletionsModel` for each model. This object takes the model name (e.g., `"gemini-pro"`) and the custom client created in the previous step.
    5.  **Assign to Agents:** When creating an agent, pass the corresponding `OpenAIChatCompletionsModel` object to the `model` parameter.

*   **Example from the Lab:** The three sales agents were configured to use different models:
    *   Agent 1: DeepSeek
    *   Agent 2: Gemini
    *   Agent 3: Llama 3 (via Grok's fast inference platform)

*   **Note on Anthropic's Claude:** It was noted that Claude does not currently offer an OpenAI-compatible endpoint, making direct integration difficult. Workarounds include using third-party services like **Open Router** or waiting for future native support.

---

#### **3. Agent Instability and Asynchronous Execution**

When running the multi-model SDR system, an important practical lesson emerged.

*   **Non-Determinism:** The first run got stuck in a loop, taking several minutes and repeatedly calling the sales agents. This was attributed to the manager agent's instruction: *"You can use the tools multiple times if you're not satisfied with the results."*
*   **Key Takeaway:** This highlights the inherent **instability and non-deterministic nature** of autonomous agents. Developers must be aware of and explicitly code for potential infinite loops.
*   **Parallel Execution:** The traces showed that the calls to the three different sales agents (DeepSeek, Gemini, Llama 3) were executed in **parallel** thanks to the framework's use of `asyncio`.

---

#### **4. Structured Outputs**

This feature allows you to force an agent to respond with a structured object (like a JSON) instead of plain text, making the output more reliable and easier to parse.

*   **How it Works:**
    1.  Define a **Pydantic class** that specifies the exact schema of the desired output. For example, a `NameCheckOutput` class was created with two fields: `is_name_in_message: bool` and `name: str`.
    2.  When creating an agent, pass this Pydantic class to the `output_type` parameter.
*   **Benefit:** This eliminates the need for fragile string parsing and ensures the agent's output is predictable and programmatically usable.

---

#### **5. Guardrails for Agent Safety**

Guardrails are safety checks to control agent inputs and outputs, preventing inappropriate content or behavior.

*   **Core Concept:** A key feature is that **guardrails can be agents themselves**, allowing the use of an LLM to perform complex validation checks.
*   **Placement:** Guardrails can only be applied at the two ends of the agent workflow:
    *   **Input Guardrails:** Check the initial user prompt before the first agent begins processing.
    *   **Output Guardrails:** Check the final result before it is returned to the user.
*   **Implementation Example (Input Guardrail):**
    1.  **Guardrail Agent:** An agent was created specifically to detect personal names in an input string. Its `output_type` was set to the `NameCheckOutput` Pydantic class (demonstrating the combination of structured outputs and guardrails).
    2.  **Guardrail Function:** An `async` function was decorated with `@input_guardrail`.
    3.  **Logic:** Inside this function, the guardrail agent was run on the input message. Based on the structured output from the agent, the function returns a `GuardrailFunctionOutput` object. If a name was detected, the object's `tripwire_triggered` attribute was set to `True`.
*   **Behavior:** When the tripwire is triggered, the framework raises an exception (`GuardrailInputTriggered`), stopping the entire process.
*   **Demonstration:**
    *   **Test 1 (Failure):** A prompt including the name "Alice" was used. The guardrail correctly identified the name, triggered the tripwire, and the execution was halted with an exception.
    *   **Test 2 (Success):** A prompt without a personal name was used. The guardrail passed, and the agent system ran successfully to completion.

---

#### **Day 3 Challenges and Next Steps**

The session concluded with exercises to solidify the concepts learned:
1.  **Experiment with more models** using the OpenAI-compatible endpoint method.
2.  **Add more guardrails,** including an **output guardrail** (e.g., to check for outdated information like "Copyright 2023").
3.  **Implement structured outputs** for the main email generation process, creating a Pydantic `Email` class with fields like `subject`, `body`, and `recipient`.

The next project was teased: **building a deep research agent**.